In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import classification_report, f1_score, recall_score, accuracy_score, precision_score,  confusion_matrix, roc_auc_score, roc_curve, auc

In [8]:
data = pd.read_csv('data/UCI_Credit_Card.csv')

In [9]:
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [10]:
data.shape

(30000, 25)

In [12]:
data.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [11]:
data['default.payment.next.month'].value_counts()

default.payment.next.month
0    23364
1     6636
Name: count, dtype: int64

# Informations personnelles du client
- ID : Identifiant unique de chaque client.

- LIMIT_BAL : Montant total du crédit accordé (en dollars NT). Cela inclut les lignes de crédit individuelles, familiales ou supplémentaires.

- SEXE : Sexe du client :

    1 = homme

    2 = femme

- ÉDUCATION : Niveau d'éducation du client :

    1 = études supérieures (graduate school)

    2 = université

    3 = lycée

    4 = autre

    5 & 6 = inconnu ou non spécifié

- MARIAGE : État matrimonial :

    1 = marié(e)

    2 = célibataire

    3 = autres (divorcé(e), séparé(e), etc.)

- ÂGE : Âge du client en années.

# Historique de remboursement (PAY_X)
    Ces variables indiquent la situation de remboursement pour chacun des 6 derniers mois (d’avril à septembre 2005) :

- PAY_0 : Statut en septembre 2005

- PAY_2 : Statut en août 2005

- PAY_3 : Statut en juillet 2005

- PAY_4 : Statut en juin 2005

- PAY_5 : Statut en mai 2005

- PAY_6 : Statut en avril 2005

Valeurs possibles :

    -1 = payé à temps

    0 = paiement effectué à l’échéance

    1 à 8 = nombre de mois de retard

    9 = 9 mois ou plus de retard

# Montants des relevés de factures (BILL_AMT_X)
    Ces variables donnent le montant dû à la fin de chaque mois :

- BILL_AMT1 : Septembre 2005

- BILL_AMT2 : Août 2005

- BILL_AMT3 : Juillet 2005

- BILL_AMT4 : Juin 2005

- BILL_AMT5 : Mai 2005

- BILL_AMT6 : Avril 2005

# Montants des paiements effectués (PAY_AMT_X)
 Ces colonnes indiquent combien le client a réellement payé pendant chacun de ces mois :

- PAY_AMT1 : Paiement en septembre 2005

- PAY_AMT2 : Paiement en août 2005

- PAY_AMT3 : Paiement en juillet 2005

- PAY_AMT4 : Paiement en juin 2005

- PAY_AMT5 : Paiement en mai 2005

- PAY_AMT6 : Paiement en avril 2005

# Variable cible
    default.payment.next.month : Indicateur si le client a fait défaut de paiement le mois suivant :

- 1 = le client n’a pas payé (défaut)

- 0 = le client a payé normalement